# Import Databases

In [2]:
# Load libraries
import riptide
import cobra
import pandas as pd
import numpy as np

# Load & Format TPM File
#### Once files are made correctly, these steps do not need to be repeated. 
#### To re-run, uncomment lines with single "#"

In [ ]:
## Load TPM data
## Replace file path(s) accordingly
#df = pd.read_csv("/Users/kathleenfurtado/RIPTIDE/TPMs_PATRIC_ID.csv", sep=',')

In [ ]:
## Format IDs (remove 'fig|')
#df['PATRIC.ID'] = df['PATRIC.ID'].str.split('|', expand=True)[1]

In [ ]:
#df.shape 
## 3400 genes  
## need to filter down to just the metabolic genes present in the model

In [ ]:
## Get the genes from model
#modelGenes = {g.id for g in r20291.genes} #703 genes

In [ ]:
## Filter TPM df by modelGenes
#df1 = df.loc[df['PATRIC.ID'].isin(modelGenes)] # 678 genes (intersection of modelGenes and TPM genes)

In [ ]:
## Separate TPM file into condition-specific files
#df1.iloc[:, :4].to_csv('TPM_CDMM.csv',sep=',',index=False) # CDMM; isolates first three columns + row names
#df1.iloc[:, [0,4,5,6]].to_csv('TPM_Comm.csv',sep=',',index=False) # Comm; isolates columns 4-6 + row names
#df1.iloc[:, [0,7,8,9]].to_csv('TPM_IEC.csv',sep=',',index=False) # IEC; isolates columns 7-9 + row names

# Make Transcriptome Dictionaries

In [3]:
# Create transcriptomic dictionaries
cdmm = riptide.read_transcription_file('TPM_CDMM.csv', norm = False, header = True, sep=',')#TPM normalized, hence norm=False
iec = riptide.read_transcription_file('TPM_IEC.csv', norm = False, header = True, sep=',')

# Analyses for Complete CDMM

## Load Model for Complete CDMM (No Mucus)

In [5]:
##Media formulation for CDMM with all amino acids

#Reload model
r20291=cobra.io.read_sbml_model("/Users/kathleenfurtado/RIPTIDE/iCdR703_gh.sbml")

cdmm_med = {'cpd00001_e', # water
        'cpd00104_e', # Biotin
        'cpd00644_e', # Pantothenate
        'cpd00263_e', # Pyridoxine
        'cpd00149_e', # Cobalt
        'cpd00099_e', # Chloride
        'cpd10515_e', # Iron
        'cpd00030_e', # Manganese
        'cpd00254_e', # Magnesium
        'cpd00063_e', # Calcium
        'cpd00205_e', # Potassium
        'cpd00009_e', # Phosphate
        'cpd00971_e', # Sodium
        'cpd00242_e', # Carbonate
        'cpd00027_e', # Glucose
        'cpd00013_e', # Ammonia
        'cpd00322_e', # L-Isoleucine
        'cpd00129_e', # L-Proline
        'cpd00156_e', # L-Valine
        'cpd00107_e', # L-Leucine
        'cpd00084_e', # L-Cysteine
        'cpd00065_e', # L-Tryptophan
        'cpd00039_e', # L-Lysine
        'cpd00119_e', # L-Histidine
        'cpd00051_e', # L-Arginine
        'cpd00041_e', # L-Aspartate
        'cpd00161_e', # L-Threonine
        'cpd00054_e', # L-Serine
        'cpd00023_e', # L-Glutamate
        'cpd00033_e', # L-Glycine
        'cpd00035_e', # L-Alanine
        'cpd00060_e', # L-Methionine
        'cpd00069_e', # L-Tyrosine
        'cpd00066_e', # L-Phenylalanine
        'cpd00053_e', # L-Glutamine
        'cpd00132_e'} # L-Asparagine

# minimal media exchange reactions
cdmm_ex = {'EX_' + x for x in cdmm_med}

# Turn off all exchanges unless in minimal media
for i in r20291.exchanges:
    i.upper_bound = 1000
    if not i.id in cdmm_ex:
        i.lower_bound = 0.
    else: 
        i.lower_bound = -1000.
        
# Check R20291 growth in CDMM
r20291.slim_optimize()

74.3745987238828

## Run RIPTIDE for No Mucus

In [6]:
# Create Riptide object
rip_cdmm = riptide.contextualize(model=r20291, conservative=False, transcriptome=cdmm)
rip_cdmm_max = riptide.maxfit_contextualize(model=r20291, conservative=False, transcriptome=cdmm) #used in paper


Initializing model and integrating transcriptomic data...
Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 324 from 1313 (75.32% change)
Metabolites pruned to 325 from 1243 (73.85% change)
Flux through the objective DECREASED to ~58.68 from ~74.37 (21.1% change)
Context-specific metabolism does not correlate with transcriptome (r=0.063, n.s.)

RIPTiDe completed in, 1 minute and 46 seconds 


Running max fit RIPTiDe for objective fraction range: 0.35 to 0.95 with intervals of 0.05 

Testing minimum objective fractions...
Fraction = 0.35 | Rho = 0.1101 ; p = 0.0577
Fraction = 0.4 | Rho = 0.0759 ; p = 0.1912
Fraction = 0.45 | Rho = 0.0754 ; p = 0.194
Fraction = 0.5 | Rho = 0.034 ; p = 0.5586
Fraction = 0.55 | Rho = 0.0897 ; p = 0.1109
Fraction = 0.6 | Rho = 0.0073 ; p = 0.8969
Fraction = 0.65 | Rho = 0.0347 ; p = 0.5362
Fraction = 0.7 | Rho = 0.0611 ; p = 0.2724
Fraction = 0.75 | Rho = 0.0641 ; p = 0.2495
Fraction = 0.8 | Rho = 0.0625

In [7]:
#Export output from CDMM condition
riptide.save_output(riptide_obj=rip_cdmm, path='/Users/kathleenfurtado/RIPTIDE/riptide_cdmm_complete')
riptide.save_output(riptide_obj=rip_cdmm_max, path='/Users/kathleenfurtado/RIPTIDE/riptide_cdmm_max_complete')

/Users/kathleenfurtado/anaconda3/envs/riptide3/lib/python3.8/site-packages/jupyter_client/jsonutil.py:75: DeprecationWarning: Interpreting naive datetime as local 2023-12-30 18:21:38.234462. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [8]:
#Check how many feasible samples
print(rip_cdmm.flux_samples.shape)
print(rip_cdmm_max.flux_samples.shape)

(348, 324)
(500, 298)


/Users/kathleenfurtado/anaconda3/envs/riptide3/lib/python3.8/site-packages/jupyter_client/jsonutil.py:75: DeprecationWarning: Interpreting naive datetime as local 2024-01-21 21:07:16.453688. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


## Load Model for Complete CDMM +Mucus

In [10]:
##Load model and edit for CDMM media with monosaccharides derived from N- and O-glycans
#Using all amino acids

#Reload model
r20291=cobra.io.read_sbml_model("/Users/kathleenfurtado/RIPTIDE/iCdR703_gh.sbml")

cdmm_muc = {'cpd00001_e', # water
        'cpd00104_e', # Biotin
        'cpd00644_e', # Pantothenate
        'cpd00263_e', # Pyridoxine
        'cpd00149_e', # Cobalt
        'cpd00099_e', # Chloride
        'cpd10515_e', # Iron
        'cpd00030_e', # Manganese
        'cpd00254_e', # Magnesium
        'cpd00063_e', # Calcium
        'cpd00205_e', # Potassium
        'cpd00009_e', # Phosphate
        'cpd00971_e', # Sodium
        'cpd00242_e', # Carbonate
        'cpd00013_e', # Ammonia
        'cpd00027_e', # D-Glucose
        'cpd00322_e', # L-Isoleucine
        'cpd00129_e', # L-Proline
        'cpd00156_e', # L-Valine
        'cpd00107_e', # L-Leucine
        'cpd00084_e', # L-Cysteine
        'cpd00065_e', # L-Tryptophan
        'cpd00039_e', # L-Lysine
        'cpd00119_e', # L-Histidine
        'cpd00051_e', # L-Arginine
        'cpd00041_e', # L-Aspartate
        'cpd00161_e', # L-Threonine
        'cpd00054_e', # L-Serine
        'cpd00023_e', # L-Glutamate
        'cpd00033_e', # L-Glycine
        'cpd00035_e', # L-Alanine
        'cpd00060_e', # L-Methionine
        'cpd00069_e', # L-Tyrosine
        'cpd00066_e', # L-Phenylalanine
        'cpd00053_e', # L-Glutamine
        'cpd00132_e', # L-Asparagine
        'cpd00122_e', # N-Acetyl-D-glucosamine
        'cpd27607_e', # N-Acetyl-D-galactosamine
        'cpd00232_e', # Neu5Ac
        'cpd00108_e', # Galactose
        'cpd00751_e', # L-Fucose
        'cpd00138_e'} # D-Mannose

# minimal media exchange reactions
cdmm_ex = {'EX_' + x for x in cdmm_muc}

# Turn off all exchanges unless in minimal media
for i in r20291.exchanges:
    i.upper_bound = 1000
    if not i.id in cdmm_ex:
        i.lower_bound = 0.
    else: 
        i.lower_bound = -1000.
        
# Check R20291 growth in CDMM
r20291.slim_optimize()

/Users/kathleenfurtado/anaconda3/envs/riptide3/lib/python3.8/site-packages/jupyter_client/jsonutil.py:75: DeprecationWarning: Interpreting naive datetime as local 2024-01-21 21:07:35.722154. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


84.62195562073408

## Run RIPTIDE for  +Mucus

In [11]:
#Create RIPTIDE Objects
rip_iec = riptide.contextualize(model=r20291, conservative=False, transcriptome=iec)
rip_iec_max = riptide.maxfit_contextualize(model=r20291, conservative=False, transcriptome=iec)

/Users/kathleenfurtado/anaconda3/envs/riptide3/lib/python3.8/site-packages/jupyter_client/jsonutil.py:75: DeprecationWarning: Interpreting naive datetime as local 2023-12-30 18:35:36.051312. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)



Initializing model and integrating transcriptomic data...
Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 341 from 1313 (74.03% change)
Metabolites pruned to 336 from 1243 (72.97% change)
Flux through the objective DECREASED to ~72.03 from ~84.62 (14.88% change)
Context-specific metabolism does not correlate with transcriptome (r=0.004, n.s.)

RIPTiDe completed in, 1 minute and 14 seconds 


Running max fit RIPTiDe for objective fraction range: 0.35 to 0.95 with intervals of 0.05 

Testing minimum objective fractions...
Fraction = 0.35 | Rho = 0.0363 ; p = 0.5214
Fraction = 0.4 | Rho = 0.0351 ; p = 0.5355
Fraction = 0.45 | Rho = 0.0009 ; p = 0.9867
Fraction = 0.5 | Rho = 0.036 ; p = 0.5116
Fraction = 0.55 | Rho = 0.0572 ; p = 0.2991
Fraction = 0.6 | Rho = 0.0002 ; p = 0.997
Fraction = 0.65 | Rho = 0.0055 ; p = 0.9192
Fraction = 0.7 | Rho = -0.001 ; p = 0.9851
Fraction = 0.75 | Rho = 0.0068 ; p = 0.9
Fraction = 0.8 | Rho = 0.0042 ;

In [12]:
#Save Outputs
riptide.save_output(riptide_obj=rip_iec, path='/Users/kathleenfurtado/RIPTIDE/riptide_iec_complete')
riptide.save_output(riptide_obj=rip_iec_max, path='/Users/kathleenfurtado/RIPTIDE/riptide_iec_max_complete')

/Users/kathleenfurtado/anaconda3/envs/riptide3/lib/python3.8/site-packages/jupyter_client/jsonutil.py:75: DeprecationWarning: Interpreting naive datetime as local 2023-12-30 19:01:58.375317. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [13]:
#Check feasible samplings per condition
print(rip_iec.flux_samples.shape) #287 samplings, 324 rxns
print(rip_iec_max.flux_samples.shape) #500 samplings, 341 rxns

(500, 341)
(500, 335)


/Users/kathleenfurtado/anaconda3/envs/riptide3/lib/python3.8/site-packages/jupyter_client/jsonutil.py:75: DeprecationWarning: Interpreting naive datetime as local 2023-12-30 19:02:21.827203. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


## Test Complete CDMM +Mucus formulation, to see if this alters biomass/feasible solutions for no mucus condition.

In [14]:
#Create RIPTIDE Objects
rip_cdmm_monosacc_test = riptide.contextualize(model=r20291, conservative=False, transcriptome=cdmm)
rip_cdmm_monosacc_test_max = riptide.maxfit_contextualize(model=r20291, conservative=False, transcriptome=cdmm)

/Users/kathleenfurtado/anaconda3/envs/riptide3/lib/python3.8/site-packages/jupyter_client/jsonutil.py:75: DeprecationWarning: Interpreting naive datetime as local 2023-12-30 19:02:58.759128. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)



Initializing model and integrating transcriptomic data...
Pruning zero flux subnetworks...
Analyzing context-specific flux distributions...

Reactions pruned to 342 from 1313 (73.95% change)
Metabolites pruned to 338 from 1243 (72.81% change)
Flux through the objective DECREASED to ~70.05 from ~84.62 (17.22% change)
Context-specific metabolism does not correlate with transcriptome (r=0.04, n.s.)

RIPTiDe completed in, 2 minutes and 6 seconds 


Running max fit RIPTiDe for objective fraction range: 0.35 to 0.95 with intervals of 0.05 

Testing minimum objective fractions...
Fraction = 0.35 | Rho = 0.1087 ; p = 0.0543
Fraction = 0.4 | Rho = 0.1022 ; p = 0.0705
Fraction = 0.45 | Rho = 0.0309 ; p = 0.5848
Fraction = 0.5 | Rho = 0.0993 ; p = 0.0703
Fraction = 0.55 | Rho = 0.0537 ; p = 0.3263
Fraction = 0.6 | Rho = 0.0737 ; p = 0.1752
Fraction = 0.65 | Rho = 0.0824 ; p = 0.1284
Fraction = 0.7 | Rho = 0.0731 ; p = 0.1776
Fraction = 0.75 | Rho = 0.0308 ; p = 0.5706
Fraction = 0.8 | Rho = 0.03

In [15]:
#Check feasible samplings
print(rip_cdmm_monosacc_test.flux_samples.shape) #85 samplings, 342 rxns
print(rip_cdmm_monosacc_test_max.flux_samples.shape) #500 samplings, 314 rxns

(85, 342)
(500, 314)


/Users/kathleenfurtado/anaconda3/envs/riptide3/lib/python3.8/site-packages/jupyter_client/jsonutil.py:75: DeprecationWarning: Interpreting naive datetime as local 2023-12-30 19:38:16.542419. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [16]:
#Export to check biomass and compare to conditions with mucus in R
riptide.save_output(riptide_obj=rip_cdmm_monosacc_test, path='/Users/kathleenfurtado/RIPTIDE/riptide_cdmm_monosacc_test')
riptide.save_output(riptide_obj=rip_cdmm_monosacc_test_max, path='/Users/kathleenfurtado/RIPTIDE/riptide_cdmm_monosacc_test_max')

/Users/kathleenfurtado/anaconda3/envs/riptide3/lib/python3.8/site-packages/jupyter_client/jsonutil.py:75: DeprecationWarning: Interpreting naive datetime as local 2023-12-30 19:44:10.758749. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


### End of Analysis